In [1]:
import numpy as np  # For array manipulation and mathematical operations
import pandas as pd  # For working with datasets
import matplotlib.pyplot as plt  # For data visualization
import seaborn as sns  # For enhanced data visualization
import scipy.stats  # For statistical functions

In [23]:
t_range = [3,18] #range of thickness in mm 
angle_range = [90,179] #range of angles in degrees
AR_range = [0,1] #range of edge radii AR in mm
k_ideal = 144.9 # Ideal Stiffness in N/mm 
v_total = 300*100*50 # Volume of load cell without any cutouts 
applied_f = 5700 # Applied force in N 
mutation_prob = 0.05
des_variables = ['Thickness','Angle','AR']
res_variables = ['delh','volume','k','fitness']
head = des_variables+res_variables
pop_num = 12

In [24]:
def generate_initial_designs(a, b, c, pop):
    designs_initial = []
    for _ in range(pop):  # Generate 12 arrays
        x = np.random.triangular(a[0], (a[0] + a[1]) / 2, a[1])  # Random thickness
        y = np.random.triangular(b[0], (b[0] + b[1]) / 2, b[1])  # Random angle
        z = np.random.triangular(c[0], (c[0] + c[1]) / 2, c[1])  # Random edge radii AR

        x = round(x, 2)
        y = round(y, 2)
        z = round(z, 2)

        # Empty values for new variables
        delh = None
        volume = None
        k = None
        fitness = None

        designs_initial.append([x, y, z, delh, volume, k, fitness])  # Append the array


    return designs_initial

def cost_function(force,del_h,v,a,b,k_ideal,v_ideal):
    k = force/del_h # k in [N/mm]
    f = (a*((k_ideal-k)^2)/(k_ideal^2)+b*(v/v_ideal))^-1
    return f 

def sort_list_by_last_value(lst):
    # Define a custom sorting key function to extract the last element of each sublist
    def last_value(sublist):
        return sublist[-1]  # Return the last element of the sublist
    
    # Sort the list of sublists based on the last value in each sublist (highest to lowest)
    sorted_list = sorted(lst, key=last_value, reverse=True)
    
    return sorted_list

def mutate(df,mutation_prob,t_range,angle_range,AR_range):
    for index, row in df.iterrows():
        for column in df.columns:
            # Generate a random number between 0 and 1
            rand_prob = np.random.rand()
            if rand_prob < mutation_prob:
                if column == 'thickness':
                    new_value = np.random.uniform(t_range[0], t_range[1])
                elif column == 'angle':
                    new_value = np.random.uniform(angle_range[0], angle_range[1])
                elif column == 'AR':
                    new_value = np.random.uniform(AR_range[0], AR_range[1])
                else: 
                    new_value = df.at[index, column]
                
                # Update the cell value
                df.at[index, column] = new_value
    return df

def genetic_algorithm(df,t_range,angle_range,AR_range,mutation_prob):
    # Reindex the DataFrame from 0
    df.reset_index(drop=True, inplace=True)

    # Add all fitness variables to create a fitness sum
    fitness_sum = df['fitness'].sum()

    # Calculate relative fitness for each row
    df['relative_fitness'] = df['fitness'] / fitness_sum

    # Use relative fitness as probabilities for selection
    selected_indices = np.random.choice(df.index, size=8, replace=False, p=df['relative_fitness'])

    # Select 8 designs based on their probability of selection
    selected_designs = df.loc[selected_indices]

    # Mate selected rows with a single point mutation
    children = []
    parent1 = []
    parent2 = []
    selected_pairs = set()  # Set to store selected pairs
    while len(children) < 12:
        # Randomly select a pair of indices
        pair_indices = tuple(sorted(np.random.choice(selected_designs.index, size=2, replace=False)))
        # Check if the pair is already selected
        if pair_indices not in selected_pairs:
            # Add the pair to the set of selected pairs
            selected_pairs.add(pair_indices)
            # Take Parents
            parent1 = df.loc[pair_indices[0]]
            parent2 = df.loc[pair_indices[1]]
            # Randomly decide which parent contributes which variable
            if np.random.choice([True, False]):  # 50% chance for swap
                child = [parent1['Thickness'], parent2['Angle'],parent2['AR']]
            else:
                child = [parent2['Thickness'], parent1['Angle'],parent1['AR']]
            # Append child to children list
            children.append(child)
    
    children = pd.DataFrame(children, columns=['Thickness','Angle','AR'])
    children = mutate(children,mutation_prob,t_range,angle_range,AR_range)

    return selected_designs , children


The following cell was only run in order to generate an initial population of 12 designs, with variables randomly spread throughout the respective design variable ranges.

In [ ]:
#init_design = generate_initial_designs(t_range,angle_range,AR_range,pop_num) 
#df = pd.DataFrame(init_design, columns=head, index=range(1, len(init_design) + 1))
#print(df)
#df.to_csv('parents.csv', index_label='Index')

The following cell reads in a design data frame from a CSV file. It is assumed that this data frame is organized as a [7,12] package. 
The headings for each of the columns should be 'Index' 'Thickness' 'Angle' 'Radii' 'delh' 'volume' 'k' 'fitness'.
The genetic algorithm starts by adding a new column 'relative fitness' which is calculated by dividing each designs fitness by the sum of all fitnesses.
This relative fitness is used as probability of selection for the designs as parents. 8 parents are selected to a mating pool, and 24 unique pairs are selected. 
Each of these pairs produces one child with a single point crossover. 
Following the generation of the initial children, the genes are mutated within their respecitve ranges, and at a frequency dictated by the 'mutation_prob' parameter. 

In [33]:
new_df = pd.read_csv('parents.csv', index_col='Index')
print(new_df)
selected,children = genetic_algorithm(new_df,t_range,angle_range,AR_range,mutation_prob)
print(children)
children.to_csv('children.csv')



       Thickness   Angle  Radii  delh   volume           k  fitness
Index                                                              
1          12.14  137.87   0.36   9.0   600000    633.3333   0.0850
2          11.48  157.48   0.50   4.0   580000   1425.0000   0.0127
3          10.69  164.43   0.44  20.0   100000    285.0000   0.9985
4           8.40  157.13   0.70  30.0   450000    190.0000   2.5197
5           9.83  101.72   0.56   6.0  1200000    950.0000   0.0316
6           9.83  148.34   0.14   9.0   150000    633.3333   0.0872
7           8.67  151.23   0.53  15.0   600000    380.0000   0.3298
8           6.20  121.34   0.41   0.4   750000  14250.0000   0.0001
9           9.96  155.17   0.69  13.0    15000    438.4615   0.2430
10          9.31  164.52   0.43  18.0  1200000    316.6667   0.4535
11          7.42  120.07   0.04  12.0  1125000    475.0000   0.1684
12         14.84  130.82   0.53   4.0   450000   1425.0000   0.0128
    Thickness   Angle  Radii
0        9.96  157.

Procedure for testing

1) Run initial design generation and paste designs into the sheet on the drive.

2) Make one of the designs in Catia by tweaking parameters.

3) Save Catia part and upload to Abacus for simulation.

4) Remesh part for simulation.

5) Run simulation.

6) Take the average change in height, and the volume of the part.

7) Repeat steps 2-6 until all delH and V's are recorded.

8) Enter delH and V's into corresponding generation sheet on drive, sheet will automatically populate with 'k' and 'fitness'.

9) Coppy and paste full dataframe from sheet into local csv file.

10) Run genetic code using local CSV.

11) The code will output the childen designs onto the second sheet locally. 

12) Copy and paste the local children data frame into next generation sheet on drive. 

13) Repeat steps 2-12 until the end of the optimization. 